A lot of this stuff hinges on having the right initial data. We cant run the agents at this moment. I Added the stuff here that was already in workshop 4, but like I said, it will not run.

Personally, I believe that we have to store the eventual data of every script in a csv file. This script needs a program of requirements.

Load Libraries

In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import pandas as pd
import numpy as np
import functions
import networkx as nx
from sklearn.cluster import KMeans
np.random.seed(0)

Define Stencil

In [2]:
# creating neighborhood definition
stencil = tg.create_stencil("von_neumann", 1, 1)
# setting the center to zero
stencil.set_index([0,0,0], 0)
print(stencil)

[[[0 0 0]
  [0 1 0]
  [0 0 0]]

 [[0 1 0]
  [1 0 1]
  [0 1 0]]

 [[0 0 0]
  [0 1 0]
  [0 0 0]]]


Define the environment

In [3]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/meshes/useable_lattice.csv')
avail_lattice = tg.lattice_from_csv(lattice_path)
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice), avail_lattice)

load program (insert the program of requirements here)

In [4]:

#loading the relative relations excell
relative_rels = pd.read_excel('../data/relationships/relative_relations.xlsx') 
#relative_rels = relative_rels.iloc[:2,:3]

#loading the relative preferences excell
relative_prefs = pd.read_excel('../data/relationships/relative_preferences.xlsx')
functions.normalize_excel(relative_prefs)
#relative_prefs = relative_prefs.head(2)

# read agent size
agent_sizes = pd.read_excel('../data/relationships/agent_size.xlsx')
#agent_sizes = agent_sizes.head(2)refs_norm.head(3)

In [5]:
program_prefs = relative_prefs.drop(["space_name"], 1)
program_rels = relative_rels.drop(["Unnamed: 0"], 1)

Load the value fields

In [6]:
# loading the lattice from csv
fields = {}
for f in program_prefs.columns:
    lattice_path = os.path.relpath('../data/fields/' + f + '.csv')
    fields[f] = tg.lattice_from_csv(lattice_path) * avail_lattice


In [7]:
agent_pref_fields=[]
for i in range(0,len(program_prefs)):
    int_d=1
    for f in program_prefs:
        t=fields[f]**program_prefs[f][i]
        int_d=int_d*t
    agent_pref_fields.append(int_d)


Initialize the agents

In [8]:
# initialize the occupation lattice
occ_lattice = avail_lattice * 0 - 1

# Finding the index of the available voxels in avail_lattice
avail_flat = avail_lattice.flatten()
avail_index = np.array(np.where(avail_lattice == 1)).T

# Randomly choosing three available voxels
# agn_num = len(program_prefs)
# np.random.seed()
# select_id = np.random.choice(len(avail_index), agn_num)
# agn_origins = avail_index[select_id]

agn_num = len(program_prefs)
agn_origins=[]
for i in range(0,agn_num):
    best_index = np.array(np.where(agent_pref_fields[i] == agent_pref_fields[i].max())).T
    best_index=best_index[0]
    agn_origins.append(best_index)
agn_origins=np.stack(agn_origins) 

# adding the origins to the agents locations
agn_locs = []
agn_vals = []
# for each agent origin ... 
for a_id, a_origin in enumerate(agn_origins):

    # add the origin to the list of agent locations
    agn_locs.append([a_origin])
    
    # set the origin in availablity lattice as 0 (UNavailable)
    avail_lattice[tuple(a_origin)] = 0

    # set the origin in occupation lattice as the agent id (a_id)
    occ_lattice[tuple(a_origin)] = a_id
    
#initializing the agents' values    
for a_id, a_prefs in program_prefs.iterrows():
    agn_loc = agn_locs[a_id]
    a_eval = 1
    for f in program_prefs.columns:        
        # find the raw value of free neighbours...
        val = fields[f][agn_loc[0][0], agn_loc[0][1], agn_loc[0][2]]
        # raise the the raw value to the power of preference weight of the agent
        a_weighted_vals = val ** a_prefs[f]
        # multiply them to the previous weighted values
        a_eval *= a_weighted_vals
    agn_vals.append([a_eval])



printing the ammount of agents

In [9]:
for a_id, a_prefs in program_prefs.iterrows():
    # create a lattice with only the agent locations enabled
    agn_lattice = functions.enabling_loc_in_lattice(agn_locs[a_id], avail_lattice)
    # calculate distance field to the agent locations through breath-first traversal
    distance_lattice = functions.breath_first_traversal(agn_lattice, avail_lattice)
    # invert the field
    fields[a_id] = 1 - functions.min_max_scaler(distance_lattice)


In [10]:
agn_origins

array([[22,  5,  7],
       [30, 15,  1],
       [ 9,  5,  7],
       [30, 15,  5],
       [26, 18,  1],
       [29, 15,  4],
       [29,  7,  1],
       [28, 14,  6],
       [33, 18,  1],
       [ 3, 15,  1],
       [ 3, 17,  1],
       [ 5,  6,  1],
       [31,  5,  1],
       [ 2, 15,  1],
       [ 2,  3,  1],
       [ 4,  5,  1],
       [33, 18,  5],
       [ 4, 17,  1],
       [29,  7,  7]], dtype=int64)

show the agents

In [11]:
p = pv.Plotter(notebook=True)

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(occ_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = occ_lattice.minbound - occ_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = occ_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#777777")

# Add the data values to the cell data
grid.cell_arrays["Agents"] = occ_lattice.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([-0.1, agn_num - 0.9])
# adding the voxels
p.add_mesh(threshed, show_edges=True, opacity=1.0, show_scalar_bar=False)

# adding the availability lattice
#init_avail_lattice.fast_vis(p)

p.show(use_ipyvtk=True)


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(235.5797258158867, 141.07972438537521, 217.47972465955658),
 (35.70000076293945, -58.80000066757202, 17.599999606609344),
 (0.0, 0.0, 1.0)]

In [12]:
# extract the address of all occupied voxels
occ_ind = np.array(np.where(occ_lattice > -1)).T

# construct kmeans model and fit it to find the clustering
kmeans_model = KMeans(n_clusters=5, random_state=0).fit(occ_ind)

In [13]:
# extract cluster centers
cluster_centers = np.round(kmeans_model.cluster_centers_).astype(np.int8)
# init shaft lattice
shft_lattice = occ_lattice * 0
# set the shafts
for cl_cen in cluster_centers:
    shft_lattice[cl_cen[0], cl_cen[1],:] = 1
shft_lattice = shft_lattice * avail_lattice

In [14]:
p = pv.Plotter(notebook=True)

base_lattice = shft_lattice

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the availability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")


# Add the data values to the cell data
grid.cell_arrays["Agents"] = base_lattice.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([0.9, 1.1])
# adding the voxels
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)

# Add the data values to the cell data
grid.cell_arrays["Agents"] = occ_lattice.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([-0.1, agn_num - 0.9])
# adding the voxels
p.add_mesh(threshed, show_edges=True, opacity=1.0, show_scalar_bar=False)

# p.add_slider_widget(create_mesh, [0, n_frames], title='Time', value=0, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(235.5797258158867, 141.07972438537521, 217.47972465955658),
 (35.70000076293945, -58.80000066757202, 17.599999606609344),
 (0.0, 0.0, 1.0)]

In [15]:
# find the number of all voxels
vox_count = avail_lattice.size 

# initialize the adjacency matrix
adj_mtrx = np.zeros((vox_count,vox_count))

# Finding the index of the available voxels in avail_lattice
avail_index = np.array(np.where(avail_lattice == 1)).T

# fill the adjacency matrix using the list of all neighbours
for vox_loc in avail_index:
    # find the 1D id
    vox_id = np.ravel_multi_index(vox_loc, avail_lattice.shape)
    # retrieve the list of neighbours of the voxel based on the stencil
    vox_neighs = avail_lattice.find_neighbours_masked(stencil, loc = vox_loc)
    # iterating over the neighbours
    for neigh in vox_neighs:
        # setting the entry to one
        adj_mtrx[vox_id, neigh] = 1.0

# construct the graph 
g = nx.from_numpy_array(adj_mtrx)

In [16]:
# initialize corridor lattice
cor_lattice = shft_lattice * 0
cor_flat = cor_lattice.flatten()
# for each voxel that needs to have access to shafts
for a_vox in occ_ind:
    
    # slice the corridor lattice horizontally
    cor_floor = shft_lattice[:,:,a_vox[2]]
    # find the vertical shaft voxel indices
    shaft_vox_inds = np.array(np.where(cor_floor > 0)).T
    paths = []
    path_lens = []
    for shft_ind in shaft_vox_inds:
        # construct the destination address
        dst_vox = np.array([shft_ind[0],shft_ind[1],a_vox[2]])
        # construct 1-dimensional indices
        src_ind = np.ravel_multi_index(a_vox, shft_lattice.shape)
        dst_ind = np.ravel_multi_index(dst_vox, shft_lattice.shape)
        # find the shortest path
        path = nx.algorithms.shortest_paths.astar.astar_path(g, src_ind, dst_ind)
        paths.append(path)
        path_lens.append(len(path))
    if(len(path_lens) == 0): 
        continue
    # find the shortest path
    shortest_path = paths[np.array(path_lens).argmin()]

    # set the shortest path occupied in the 
    cor_flat[shortest_path] = 1

# reshape the flat lattice
cor_lattice = cor_flat.reshape(cor_lattice.shape)

In [17]:
p = pv.Plotter(notebook=True)

base_lattice = shft_lattice + cor_lattice

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the availability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")


# Add the data values to the cell data
grid.cell_arrays["Agents"] = base_lattice.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([0.9, 2.1])
# adding the voxels
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)

# Add the data values to the cell data
grid.cell_arrays["Agents"] = occ_lattice.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([-0.1, agn_num - 0.9])
# adding the voxels
p.add_mesh(threshed, show_edges=True, opacity=1.0, show_scalar_bar=False)

# p.add_slider_widget(create_mesh, [0, n_frames], title='Time', value=0, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(235.5797258158867, 141.07972438537521, 217.47972465955658),
 (35.70000076293945, -58.80000066757202, 17.599999606609344),
 (0.0, 0.0, 1.0)]

Run the simulation

In [18]:
# make a deep copy of occupation lattice
cur_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)
# initialzing the list of frames
frames = [cur_occ_lattice]

# setting the time variable to 0
t = 0
n_frames = 380
# main feedback loop of the simulation (for each time step ...)
while t<n_frames:
    # for each agent ... 
    for a_id, a_prefs in program_prefs.iterrows():
        # retrieve the list of the locations of the current agent
        a_locs = agn_locs[a_id]
        a_vals = agn_vals[a_id]

        #get max building depth for the current agentS
        max_depth = 3#agent_sizes.loc[a_id, 'voxel_depth']
        max_height = 2
        # initialize the list of free neighbours
        free_neighs = []

        agent_lattice = functions.enabling_loc_in_lattice(a_locs, avail_lattice)

        # for each location of the agent
        for loc in a_locs:
            # retrieve the list of neighbours of the agent based on the stencil
            neighs = avail_lattice.find_neighbours_masked(stencil, loc = loc)

            #BUILDING DEPTH AND BUILDING HEIGHT LOGIC
            #calculate index that the agent should avoid to achieve max building depth
            index_to_avoid_building_depth = functions.index_to_avoid_for_building_depth(agent_lattice, max_depth)
            #calculate index that the agent should avoid to achieve max building height
            index_to_avoid_building_height = functions.index_to_avoid_for_building_height(agent_lattice, max_height)
            
            # for each neighbour ... 
            for n in neighs:
                # compute 3D index of neighbour
                neigh_3d_id = np.unravel_index(n, avail_lattice.shape)
                # if the neighbour is available and avoidance logic to achieve max building depth
                if avail_lattice[neigh_3d_id] and functions.array_element_not_in_2d_array(neigh_3d_id, index_to_avoid_building_depth) and functions.array_element_not_in_2d_array(neigh_3d_id, index_to_avoid_building_height):
                    # add the neighbour to the list of free neighbours
                    free_neighs.append(neigh_3d_id)
        # check if found any free neighbour
        if len(free_neighs)>0:
            # convert free neighbours to a numpy array
            fns = np.array(free_neighs)
            
            # find the value of neighbours
            # init the agent value array
            a_eval = np.ones(len(fns))
            # for each field...
            for f in program_prefs.columns:
                # find the raw value of free neighbours...
                vals = fields[f][fns[:,0], fns[:,1], fns[:,2]]
                # raise the the raw value to the power of preference weight of the agent
                a_weighted_vals = vals ** a_prefs[f]
                # multiply them to the previous weighted values
                a_eval *= a_weighted_vals

            #post-processing
            functions.squareness(square_weight=0.2, free_neighs=free_neighs, a_eval=a_eval)
            

            #evaluating agent connectiveness
            for r in range(program_rels.columns.size):
                vals = fields[r][fns[:,0], fns[:,1], fns[:,2]]
                a_weighted_vals = vals ** program_rels.iloc[a_id][r]
                a_eval *= a_weighted_vals

            # if the agents have reached their max size -> remove the worst voxel
            if len(a_locs) >= agent_sizes.loc[a_id, 'voxel_total'] and np.max(a_eval) > np.min(a_vals): 
                smallest_voxel_val_id = np.argmin(a_vals)
                voxel_to_be_replaced_id = tuple(a_locs[smallest_voxel_val_id])

                #remove the worst voxel
                agn_locs[a_id].pop(smallest_voxel_val_id)
                agn_vals[a_id].pop(smallest_voxel_val_id)

                #voxel that is removed will be available and unoccupied
                avail_lattice[voxel_to_be_replaced_id] = 1
                occ_lattice[voxel_to_be_replaced_id] = -1

            # re-evaluate the agent size (since the worst voxel may have been removed in the last condition check)
            # if it is smaller then max size -> add best voxel    
            if len(a_locs) < agent_sizes.loc[a_id, 'voxel_total']:
                # select the neighbour with highest evaluation
                selected_int = np.argmax(a_eval)
                # find 3D integer index of selected neighbour
                selected_neigh_3d_id = free_neighs[selected_int]
                # find the location of the newly selected neighbour
                selected_neigh_loc = np.array(selected_neigh_3d_id).flatten()
                # add the newly selected neighbour location to agent locations
                agn_locs[a_id].append(selected_neigh_loc)
                # add the newly best value selected
                agn_vals[a_id].append(a_eval[selected_int])
                # set the newly selected neighbour as UNavailable (0) in the availability lattice
                avail_lattice[selected_neigh_3d_id] = 0
                # set the newly selected neighbour as OCCUPIED by current agent 
                # (-1 means not-occupied so a_id)
                occ_lattice[selected_neigh_3d_id] = a_id
            
            # updating the distance field for agent connectiveness
            # lattice with only the current agent location enabled
            agn_lattice = functions.enabling_loc_in_lattice(agn_locs[a_id], avail_lattice)
            # calculate new distance lattice
            distance_lattice = functions.breath_first_traversal(agn_lattice, avail_lattice)
            # invert the distance
            fields[a_id] = 1 - functions.min_max_scaler(distance_lattice)

  
    # constructing the new lattice
    new_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)
    # adding the new lattice to the list of frames
    frames.append(new_occ_lattice)
    # adding one to the time counter
    print(t)
    t += 1 


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

Visualize the result

In [19]:
p = pv.Plotter(notebook=True)

base_lattice = frames[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the availability lattice
#init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    f = int(value)
    lattice = frames[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([-0.1, agn_num - 0.9])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)

    return

p.add_slider_widget(create_mesh, [0, n_frames], title='Time', value=0, event_type="always", style="classic")
p.show(use_ipyvtk=True)


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(235.5797258158867, 141.07972438537521, 217.47972465955658),
 (35.70000076293945, -58.80000066757202, 17.599999606609344),
 (0.0, 0.0, 1.0)]

In [23]:
agn_locs[18]

[array([29,  7,  7], dtype=int64),
 array([29,  8,  7], dtype=int64),
 array([29,  9,  7], dtype=int64),
 array([29, 10,  6], dtype=int64),
 array([29,  9,  6], dtype=int64),
 array([30,  9,  6], dtype=int64),
 array([30, 10,  7], dtype=int64),
 array([30, 10,  6], dtype=int64),
 array([30,  9,  7], dtype=int64),
 array([29, 11,  6], dtype=int64),
 array([30, 11,  6], dtype=int64)]

In [20]:
for i, lattice in enumerate(frames):
    csv_path = os.path.relpath('../data/abm_animation/abm_f_'+ f'{i:03}' + '.csv')
    lattice.to_csv(csv_path)
